In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator

from indra_db_lite.api import get_entrez_pmids_for_hgnc
from indra_db_lite.api import get_entrez_pmids_for_uniprot
from indra_db_lite.api import get_plaintexts_for_text_ref_ids
from indra_db_lite.api import get_text_ref_ids_for_agent_text
from indra_db_lite.api import get_text_ref_ids_for_pmids


from adeft_indra.grounding import AdeftGrounder
from adeft_indra.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename

In [2]:
def get_text_ref_ids_for_entity(ns, id_):
    if ns == 'HGNC':
        pmids = get_entrez_pmids_for_hgnc(id_)
    elif ns == 'UP':
        pmids = get_entrez_pmids_for_uniprot(id_)
    return list(get_text_ref_ids_for_pmids(pmids).values())

In [3]:
adeft_grounder = AdeftGrounder()

In [4]:
shortforms = ['RIP']
model_name = '&'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../../', 'results', model_name))

In [5]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    text_ref_ids = get_text_ref_ids_for_agent_text(shortform)
    content = get_plaintexts_for_text_ref_ids(text_ref_ids, contains=shortforms)
    text_dict = content.flatten()
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('rna immunoprecipitation', 382),
 ('receptor interacting protein', 212),
 ('regulated intramembrane proteolysis', 82),
 ('repeat induced point mutation', 77),
 ('rat insulin promoter', 52),
 ('ribosome inactivating protein', 37),
 ('protein immunoprecipitation', 30),
 ('receptor interacting protein kinase', 16),
 ('repeat induced point', 15),
 ('rnaiii inhibiting peptide', 13),
 ('ribonucleoprotein immunoprecipitation', 9),
 ('remote ischemic postconditioning', 7),
 ('respiratory inductive plethysmography', 6),
 ('remote ischemic preconditioning', 5),
 ('rna iii inhibiting peptide', 5),
 ('rna immunoprecipitation assay', 5),
 ('radix isatidis polysaccharide', 3),
 ('rna ip', 3),
 ('rat insulin promoter ii', 3),
 ('regulated intra membrane proteolysis', 3),
 ('rna immunoprecipitation chip', 3),
 ('receptor interacting serine threonine protein', 3)]

In [10]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [11]:
result = [grounding_map, names, pos_labels]

In [12]:
result

[{'protein immunoprecipitation': 'EFO:0005033',
  'radix isatidis polysaccharide': 'ungrounded',
  'rat insulin promoter': 'rat_insulin_promoter',
  'rat insulin promoter ii': 'rat_insulin_promoter',
  'receptor interacting protein': 'FPLX:RIPK',
  'receptor interacting protein kinase': 'FPLX:RIPK',
  'receptor interacting serine threonine protein': 'FPLX:RIPK',
  'regulated intra membrane proteolysis': 'ungrounded',
  'regulated intramembrane proteolysis': 'ungrounded',
  'remote ischemic postconditioning': 'MESH:D057775',
  'remote ischemic preconditioning': 'MESH:D019194',
  'repeat induced point': 'repeat_induced_point',
  'repeat induced point mutation': 'repeat_induced_point',
  'respiratory inductive plethysmography': 'MESH:D010991',
  'ribonucleoprotein immunoprecipitation': 'EFO:0005033',
  'ribosome inactivating protein': 'MESH:D000076985',
  'rna iii inhibiting peptide': 'MESH:C111711',
  'rna immunoprecipitation': 'EFO:0005033',
  'rna immunoprecipitation assay': 'EFO:00050

In [13]:
grounding_map, names, pos_labels = [{'protein immunoprecipitation': 'EFO:0005033',
  'radix isatidis polysaccharide': 'ungrounded',
  'rat insulin promoter': 'rat_insulin_promoter',
  'rat insulin promoter ii': 'rat_insulin_promoter',
  'receptor interacting protein': 'FPLX:RIPK',
  'receptor interacting protein kinase': 'FPLX:RIPK',
  'receptor interacting serine threonine protein': 'FPLX:RIPK',
  'regulated intra membrane proteolysis': 'ungrounded',
  'regulated intramembrane proteolysis': 'ungrounded',
  'remote ischemic postconditioning': 'MESH:D057775',
  'remote ischemic preconditioning': 'MESH:D019194',
  'repeat induced point': 'repeat_induced_point',
  'repeat induced point mutation': 'repeat_induced_point',
  'respiratory inductive plethysmography': 'MESH:D010991',
  'ribonucleoprotein immunoprecipitation': 'EFO:0005033',
  'ribosome inactivating protein': 'MESH:D000076985',
  'rna iii inhibiting peptide': 'MESH:C111711',
  'rna immunoprecipitation': 'EFO:0005033',
  'rna immunoprecipitation assay': 'EFO:0005033',
  'rna immunoprecipitation chip': 'EFO:0005033',
  'rna ip': 'EFO:0005033',
  'rnaiii inhibiting peptide': 'MESH:C111711'},
 {'EFO:0005033': 'RIP',
  'rat_insulin_promoter': 'rat_insulin_promoter',
  'FPLX:RIPK': 'RIPK',
  'MESH:D057775': 'Ischemic Postconditioning',
  'MESH:D019194': 'Ischemic Preconditioning',
  'repeat_induced_point': 'repeat_induced_point',
  'MESH:D010991': 'Plethysmography',
  'MESH:D000076985': 'Saporins',
  'MESH:C111711': 'RNAIII inhibiting peptide'},
 ['FPLX:RIPK', 'MESH:C111711', 'MESH:D000076985']]

In [14]:
excluded_longforms = []

In [15]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {}

In [17]:
unambiguous_agent_texts = {}

In [18]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts(
    (text, text_ref_id) for text_ref_id, text in all_texts.items()
)
agent_text_text_ref_id_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_text_ref_id_map[label].append(id_)

entity_text_ref_id_map = {
    entity: set(
        get_text_ref_ids_for_entity(*entity.split(':', maxsplit=1))
    )
    for entity in additional_entities
}

In [19]:
intersection1 = []
for entity1, trids1 in entity_text_ref_id_map.items():
    for entity2, trids2 in entity_text_ref_id_map.items():
        intersection1.append((entity1, entity2, len(trids1 & trids2)))

In [20]:
intersection2 = []
for entity1, trids1 in agent_text_text_ref_id_map.items():
    for entity2, pmids2 in entity_text_ref_id_map.items():
        intersection2.append((entity1, entity2, len(set(trids1) & trids2)))

In [21]:
intersection1

[]

In [22]:
intersection2

[]

In [23]:
all_used_trids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_trids = set()
    for agent_text in agent_texts[1]:
        trids = set(get_text_ref_ids_for_agent_text(agent_text))
        new_trids = list(trids - all_texts.keys() - used_trids)
        content = get_plaintexts_for_text_ref_ids(new_trids, contains=agent_texts[1])
        text_dict = content.flatten()
        corpus.extend(
            [
                (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
            ]
        )
        used_trids.update(new_trids)
    all_used_trids.update(used_trids)
        
for entity, trids in entity_text_ref_id_map.items():
    new_trids = list(set(trids) - all_texts.keys() - all_used_trids)
    _, contains = additional_entities[entity]
    content = get_plaintexts_for_text_ref_ids(new_trids, contains=contains)
    text_dict = content.flatten()
    corpus.extend(
        [
            (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
        ]
    )

In [24]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [25]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2021-10-09 18:38:38] /adeft/Py/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2021-10-09 18:38:50] /adeft/Py/adeft/adeft/modeling/classify.py - Best f1 score of 0.9611784976515431 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [26]:
classifier.stats

{'label_distribution': {'FPLX:RIPK': 190,
  'rat_insulin_promoter': 45,
  'EFO:0005033': 306,
  'repeat_induced_point': 53,
  'MESH:D000076985': 32,
  'ungrounded': 69,
  'MESH:D019194': 4,
  'MESH:C111711': 14,
  'MESH:D057775': 3,
  'MESH:D010991': 5},
 'f1': {'mean': 0.961178, 'std': 0.011756},
 'precision': {'mean': 0.975291, 'std': 0.023366},
 'recall': {'mean': 0.948836, 'std': 0.032127},
 'EFO:0005033': {'f1': {'mean': 0.983999, 'std': 0.011224},
  'pr': {'mean': 0.996774, 'std': 0.006452},
  'rc': {'mean': 0.971968, 'std': 0.024751}},
 'FPLX:RIPK': {'f1': {'mean': 0.968464, 'std': 0.010455},
  'pr': {'mean': 0.968421, 'std': 0.025784},
  'rc': {'mean': 0.969846, 'std': 0.028741}},
 'MESH:C111711': {'f1': {'mean': 0.96, 'std': 0.08},
  'pr': {'mean': 0.933333, 'std': 0.133333},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D000076985': {'f1': {'mean': 0.908252, 'std': 0.063808},
  'pr': {'mean': 0.838095, 'std': 0.105839},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D010991': {'f1':

In [27]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [28]:
disamb.dump(model_name, results_path)

In [29]:
print(disamb.info())

Disambiguation model for RIP

Produces the disambiguations:
	Ischemic Postconditioning	MESH:D057775
	Ischemic Preconditioning	MESH:D019194
	Plethysmography	MESH:D010991
	RIP	EFO:0005033
	RIPK*	FPLX:RIPK
	RNAIII inhibiting peptide*	MESH:C111711
	Saporins*	MESH:D000076985
	rat_insulin_promoter	rat_insulin_promoter
	repeat_induced_point	repeat_induced_point

Class level metrics:
--------------------
Grounding                	Count	F1     
                      RIP	306	  0.984
                     RIPK*	190	0.96846
               Ungrounded	 69	0.96182
     repeat_induced_point	 53	    1.0
     rat_insulin_promoter	 45	0.91567
                 Saporins*	 32	0.90825
RNAIII inhibiting peptide*	 14	   0.96
          Plethysmography	  5	    0.6
 Ischemic Preconditioning	  4	    0.2
Ischemic Postconditioning	  3	    0.6

Global Metrics:
-----------------
	F1 score:	0.96118
	Precision:	0.97529
	Recall:		0.94884

* Positive labels
See Docstring for explanation



In [30]:
model_to_s3(disamb)

/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:

In [26]:
from adeft.disambiguate import load_disambiguator

In [27]:
disamb = load_disambiguator("BAL")

In [28]:
disamb

In [29]:
print(_28.info())

Disambiguation model for BAL

Produces the disambiguations:
	Bronchoalveolar Lavage	MESH:D018893
	CEL*	HGNC:1848
	Liver, Artificial	MESH:D019164
	benzaldehyde lyase*	MESH:C059416
	betaine aldehyde*	CHEBI:CHEBI:15710
	dimercaprol*	CHEBI:CHEBI:64198

Class level metrics:
--------------------
Grounding             	Count	F1     
Bronchoalveolar Lavage	1259	 0.9929
                   CEL*	  36	    1.0
     Liver, Artificial	  18	0.83619
            Ungrounded	  17	   0.65
           dimercaprol*	   8	    0.4
    benzaldehyde lyase*	   3	    0.2
      betaine aldehyde*	   2	    0.2

Global Metrics:
-----------------
	F1 score:	0.90773
	Precision:	1.0
	Recall:		0.83293

* Positive labels
See Docstring for explanation



/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is 